## Программа Reboot 2020 поток 1 КУ Сбербанка
### выполнил Малинкин Павел Борисович

Ниже код, который можно разместить на web сервере для демонстрации работы модели классификации героев мультсериала Симпсоны.
Мне этого не удалось сделать из-за недостатка ресурсов на квоте Heroku для бесплатного использования,
поэтому направляю код с описанием

In [ ]:
# загрузка необходимых библиотек
import torch
import pickle
import numpy as np
import PIL
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.nn import functional as F
from torchvision import transforms, datasets, models
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [14]:
# Загрузим класс, который использовался для обучения модели классификации
class SimpsonsDataset(Dataset):
    def __init__(self, files, mode):
        super().__init__()
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def augment(self, aug, image):
        return aug(image=image)['image']
        
    def load_sample(self, file):
        image = Image.open(file)
        if self.mode == 'train':
            image_np = np.array(image)       
            image = Image.fromarray(self.augment(self.pil_transform(), image_np))
        #image.save('out.jpg') - для проверки полученной картинки
        return image

    def pil_transform(self, p=1):    # дополнительная функция для албументации картинок
        return Compose([
                    albm.RandomResizedCrop(256, 256),  # растянем и вырежем центр картинки
                    albm.OneOf([
                        # apply one of transforms to 50% of images
                        albm.RandomContrast(), # apply random contrast
                        albm.RandomGamma(), # apply random gamma
                        albm.RandomBrightness(), # apply random brightness
                        albm.ShiftScaleRotate(),
                        albm.ToGray(p=0.2),
                        ],
                        p = 0.5),            
                    ])     
  

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225],) 
        ])       
        
        x = self.load_sample(self.files[index])
        # в метод load_sample добавлена албументация картинок
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)

        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [4]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))
model = pickle.load(open("model_sim.pkl", 'rb'))

In [6]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Код для вызова работы модели для классификации картинки

In [7]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

In [43]:
# Здесь нужно настроить путь для загрузки картинки
from pathlib import Path
file_name = '*.jpg'
TEST_DIR = Path('./')
test_files = sorted(list(TEST_DIR.rglob('11*.jpg')))
# test_files = sorted(list(TEST_DIR.rglob(file_name)))  -- для загрузки по имени файла
test_files, test_files[0]

([WindowsPath('11-15.jpg')], WindowsPath('11-15.jpg'))

In [44]:
# Преобразуем картинку для классификации
val_dataset = SimpsonsDataset(test_files, mode='val')
ex_img, true_label = val_dataset[0]
probs_im = predict_one_sample(model, ex_img.unsqueeze(0))
probs_im

array([[6.48265187e-13, 2.58478378e-11, 5.67217350e-15, 2.46068611e-14,
        1.26102349e-11, 4.66464088e-17, 2.44028357e-11, 4.00723205e-12,
        6.37014421e-13, 1.08802695e-14, 1.00041870e-16, 2.37735803e-10,
        6.20240868e-18, 4.12101307e-15, 9.74560428e-16, 4.94226025e-13,
        6.82803380e-12, 8.67903752e-12, 1.36900300e-11, 2.69721023e-14,
        9.99999523e-01, 8.44414743e-08, 3.63426523e-07, 1.71800711e-13,
        1.28971066e-13, 7.83665667e-12, 1.50577044e-12, 6.61074459e-11,
        9.39939706e-12, 1.61960679e-14, 5.11391703e-14, 1.01770946e-12,
        1.24517974e-11, 1.34014969e-15, 9.29729369e-17, 3.01203933e-14,
        1.33565103e-11, 4.07608631e-08, 6.21267274e-11, 1.29471840e-14,
        2.49465083e-17, 2.52829252e-13]], dtype=float32)

In [45]:
# Классифицируем картинку
y_pred = np.argmax(probs_im,-1)
y_pred_ls = [label_encoder.classes_[i] for i in y_pred]
label_classes = list(label_encoder.classes_)
preds_class = [label_classes.index(i) for i in y_pred_ls]
preds_class

[20]

In [46]:
# Найдем имя персонажа
label_classes[preds_class[0]]

'lisa_simpson'